<a href="https://colab.research.google.com/github/ampham03/grad-assessment/blob/main/grad_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
from transformers import MarianMTModel, MarianTokenizer
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [4]:
model_name = "Helsinki-NLP/opus-mt-en-de"  # English to German
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

# load WMT dataset for de-en
de_en_dev = load_dataset("wmt19", "de-en", split='validation')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

train-00014-of-00016.parquet:   0%|          | 0.00/305M [00:00<?, ?B/s]

train-00015-of-00016.parquet:   0%|          | 0.00/231M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/495k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/34782245 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2998 [00:00<?, ? examples/s]

In [5]:
# tokenize a sentence
def tokenize(sentence):
  return tokenizer(sentence, return_tensors="pt", add_special_tokens=True).input_ids

In [25]:
# decode a sequence of tokens
def decode(token_ids):
  return tokenizer.decode(token_ids, skip_special_tokens=True)

In [7]:
class Hypothesis:
  def __init__(self, score=0, is_open=True, sequence=torch.tensor([[model.config.decoder_start_token_id]]), constraints=[]):
    self.score = score # keeps track of current sequence score
    self.is_open = is_open # keeps track if the hypothesis is currently open
    self.sequence = sequence # current sequence of tokens
    self.constraints = constraints # list of constraints the sequence currently contains

In [8]:
# create a grid with dimensions of the max length of a sequence and the number of constraints + 1
# (to account for when there are no constraints in the sequence)
def init_grid(max_len, num_constraints):
  grid = []
  for i in range(max_len):
    row = []
    for j in range(num_constraints + 1):
        row.append([])
    grid.append(row)
  return grid

In [9]:
constraint_boost = 11

In [59]:
# generate new open hypotheses
def generate(model, hyp, input_ids, beam_size, encoder_outputs, constraints=[]):

  # check if sequence has EOS, return since no tokens can be added
  if hyp.sequence[0, -1].item() == tokenizer.eos_token_id:
    return [hyp]

  decoder_input_ids = hyp.sequence

  with torch.no_grad():
    outputs = model(
        input_ids=None,
        encoder_outputs=encoder_outputs,
        decoder_input_ids=decoder_input_ids
    )
    logits = outputs.logits

  # get the logits for the last predicted token
  next_token_logits = logits[:, -1, :] # [batch_size, sequence_length, vocab_size]
  next_token_probs = torch.softmax(next_token_logits, dim=-1).squeeze(0)

  # get the top beam_size tokens
  top_token_probs, top_token_ids = torch.topk(next_token_probs, beam_size)

  new_hyps = []

  # create new hypotheses with the top beam_size tokens
  for i in range(beam_size):
    next_token_id = top_token_ids[i].item()
    next_token_prob = top_token_probs[i].item()

    decoded_token = tokenizer.decode(next_token_id, skip_special_tokens=False)

    # check to see if the new token is a constraint and
    # if that constraint is already in the sequence
    flag = False
    for constraint in hyp.constraints:
      if decoded_token in constraint or decoded_token == constraint:
        flag = True
        continue

    # if it is, skip the token
    if flag:
      continue

    new_constraints = hyp.constraints
    new_score = 0
    is_open = True

    # if the new token is a constraint and it is not in the sequence,
    # add it to the constraint list
    for constraint in constraints:
      curr_constraint = constraint.split() if " " in constraint else [constraint]
      if (decoded_token == curr_constraint[0]):
        new_constraints = hyp.constraints + [decoded_token]
        new_score = constraint_boost

        # if the constraint is more than one word, close the hypothesis
        # so the constraint can be continued
        if len(curr_constraint) > 1:
          is_open = False

    new_score = new_score + hyp.score + next_token_prob # add the score of the token to the total score
    new_sequence = torch.cat([decoder_input_ids, torch.tensor([[next_token_id]])], dim=-1) # add the token to the sequence

    new_hyp = Hypothesis(new_score, is_open, new_sequence, new_constraints) # create a new hypothesis
    new_hyps.append(new_hyp)

  return new_hyps


In [27]:
# start new constrained hypotheses
def start(model, hyp, input_ids, constraints, encoder_outputs):

  # check if sequence has EOS, return since no tokens can be added
  if hyp.sequence[0, -1].item() == tokenizer.eos_token_id:
    return [hyp]

  new_hyps = []

  # get the first word of a multi word constraint or a single word constraint itself
  for constraint in constraints:
    first_word = constraint.split()[0] if ' ' in constraint else constraint
    # if the constraint is already in the sequence, skip it
    if first_word in hyp.constraints:
      continue

    first_word_token_ids = tokenizer.encode(first_word, add_special_tokens=False) # get the tokens of the word

    new_sequence = hyp.sequence
    log_prob = 0.0

    # takes account for multi token words
    for token_id in first_word_token_ids:
      new_sequence = torch.cat([new_sequence, torch.tensor([[token_id]])], dim=-1) # add the token to the sequence

      with torch.no_grad():
        outputs = model(
            input_ids=None,
            encoder_outputs=encoder_outputs,
            decoder_input_ids=new_sequence
        )
        logits = outputs.logits

      # get the logits for the last predicted token
      next_token_logits = logits[:, -1, :] # [batch_size, sequence_length, vocab_size]
      log_probs = torch.log_softmax(next_token_logits, dim=-1).squeeze(0)

      log_prob += log_probs[token_id].item() # add up the probs of each token

    constraint_factor = len(hyp.constraints) if len(hyp.constraints) > 0 else 1 # normalization factor
    # log_prob / len(first_word_token_ids) to take account of multi token constraints and
    # constraint_boost / constraint_factor to make each word in the constraint equal
    new_score = (hyp.score + (log_prob / len(first_word_token_ids))) + (constraint_boost / constraint_factor)

    new_constraints = hyp.constraints + [first_word]
    is_open = False if ' ' in constraint else True # single word constraints are finished, multi word constraints need to be continued

    new_hyp = Hypothesis(new_score, is_open, new_sequence, new_constraints)
    new_hyps.append(new_hyp)

  return new_hyps

In [56]:
# continue unfinished constraints
def cont(model, hyp, input_ids, constraints, encoder_outputs):

  # check if sequence has EOS, return since no tokens can be added
  if hyp.sequence[0, -1].item() == tokenizer.eos_token_id:
    return hyp

  for constraint in constraints:
    # look at the last constraint added to the sequence
    if (hyp.constraints[-1] in constraint):
      constraint_words = constraint.split()
      completed_words = [word for word in hyp.constraints if word in constraint_words] # get the completed words of the constraint

      # check if the constraint is incomplete
      if (len(completed_words) < len(constraint_words)):
        next_word = constraint_words[len(completed_words)]
        next_word_token_ids = tokenizer.encode(next_word, add_special_tokens=False)

        new_sequence = hyp.sequence
        log_prob = 0.0

        # takes account for multi token words
        for token_id in next_word_token_ids:
          new_sequence = torch.cat([new_sequence, torch.tensor([[token_id]])], dim=-1) # add the token to the sequence

          with torch.no_grad():
            outputs = model(
                input_ids=None,
                encoder_outputs=encoder_outputs,
                decoder_input_ids=new_sequence
            )
            logits = outputs.logits

          # get the logits for the last predicted token
          next_token_logits = logits[:, -1, :] # [batch_size, sequence_length, vocab_size]
          log_probs = torch.log_softmax(next_token_logits, dim=-1).squeeze(0)

          log_prob += log_probs[token_id].item() # add up the probs of each token

        constraint_factor = len(hyp.constraints) if len(hyp.constraints) > 0 else 1 # normalization factor
        # log_prob / len(first_word_token_ids) to take account of multi token constraints and
        # constraint_boost / constraint_factor to make each word in the constraint equal
        new_score = hyp.score + (log_prob / len(next_word_token_ids)) + (constraint_boost / constraint_factor)

        new_constraints = hyp.constraints + [next_word]

        completed_words.append(next_word)
        is_open = len(completed_words) == len(constraint_words) # check if constraints are completed

        new_hyp = Hypothesis(new_score, is_open, new_sequence, new_constraints)

        return new_hyp

  return hyp

In [67]:
def constrained_beam_search(model, input_ids, constraints, max_len, num_constraints, beam_size=10):

  """
  model: model
  input_ids: tokenized sentence
  constraints: list of constraints
  max_len: max length of translation
  num_constraints: total number of words in the list of constraints
  beam_size: beam size
  """
  start_hyp = Hypothesis() # initial hypothesis
  grid = init_grid(max_len, num_constraints) # initialize beams in grid
  grid[0][0] = [start_hyp]

  # forward pass
  with torch.no_grad():
    encoder_outputs = model.get_encoder()(input_ids=input_ids)

  for t in range(1, max_len): # time step
    for c in range(max(0, (num_constraints + t) - max_len), min(t, num_constraints) + 1):
      n, s, g = [], [], []

      for hyp in grid[t - 1][c]:
        if hyp.is_open:
          # generate new hypotheses
          new_hyps = generate(model, hyp, input_ids, beam_size, encoder_outputs, constraints)
          g.extend(new_hyps)

      if c > 0:
        for hyp in grid[t - 1][c - 1]:
          if hyp.is_open:
            # start new constraints
            new_hyps = start(model, hyp, input_ids, constraints, encoder_outputs)
            s.extend(new_hyps)
          else:
            # continue unfinished constraints
            new_hyp = cont(model, hyp, input_ids, constraints, encoder_outputs)
            n.append(new_hyp)

      # sort the hypotheses from highest score to lowest score
      all_hyps = sorted(n + s + g, key=lambda hyp: hyp.score, reverse=True)

      valid_hyps = []
      # valid hyps are hyps where the sequence has not ended or all the constraints are met
      for hyp in all_hyps:
        # where the sequence has not ended means it can still be expanded
        if (hyp.sequence[0, -1].item() != tokenizer.eos_token_id):
          valid_hyps.append(hyp)
        else:
          # where the sequence has ended and it has all the constraints means it's done
          if (len(hyp.constraints) == num_constraints):
            valid_hyps.append(hyp)

      grid[t][c] = valid_hyps[:beam_size] # top beam_size scoring hypotheses stay on the beam

  top_level_hyps = []
  # get hyps in top level beams
  for t in range(len(grid)):
    top_level_hyps.extend(grid[t][num_constraints])

  finished_hyps = []
  # get hyps with EOS token
  for hyp in top_level_hyps:
    if hyp.sequence[0, -1].item() == tokenizer.eos_token_id:
      finished_hyps.append(hyp)

  best_hyp = max(finished_hyps, key=lambda hyp: hyp.score) # get the best scoring hyp
  return best_hyp


In [68]:
def generate_baseline_translation(model, input_ids):
  hyp = Hypothesis()

  # forward pass
  with torch.no_grad():
    encoder_outputs = model.get_encoder()(input_ids=input_ids)

  while True:
    hyp = generate(model, hyp, input_ids, 1, encoder_outputs)[0]

    if hyp.sequence[0, -1].item() == tokenizer.eos_token_id:
      break

  return hyp.sequence

In [14]:
sentence = "He was also an anti- smoking activist and took part in several campaigns."
input_ids = tokenize(sentence)
constraints = ['Ebenso setzte er', 'gegen das Rauchen']
num_constraints = sum(len(constraint.split()) if " " in constraint else 1 for constraint in constraints)

In [15]:
sentence = "Munich 1856: Four maps that will change your view of the city."
input_ids = tokenize(sentence)
constraints = ['Ihren Blick']
num_constraints = sum(len(constraint.split()) if " " in constraint else 1 for constraint in constraints)

In [ ]:
len(sentence.split())

12

In [63]:
sentence = de_en_dev[1]['translation']['en']
input_ids = tokenize(sentence)
constraints = ['Eine Irren-Anstalt', 'Jugendliche begegnen', 'wo sich heute']
num_constraints = sum(len(constraint.split()) if " " in constraint else 1 for constraint in constraints)

In [64]:
constrained_translation = constrained_beam_search(model, input_ids, constraints, len(sentence.split()) + 10, num_constraints)

In [37]:
baseline_translation = generate_baseline_translation(model, input_ids)

In [ ]:
constrained_translation.sequence

tensor([[58100,   201,   133,    89,  2226, 14348,    45, 10694,   651,    10,
          3549,    39,  4356, 30902,  6143,   155,  1487,  1769,   452,   239,
           227,   110,     3,     0]])

In [45]:
constrained_translation.constraints

['sich', 'heute', 'Eine', 'Irren-Anstalt', 'Jugendliche', 'begegnen']

In [65]:
decode(constrained_translation.sequence.flatten().tolist())
# Ebenso setzte er sich gegen das Rauchen ein und nahm an mehreren Kampagnen teil .

'Eine Irren-Anstalt, wo sich heute Jugendliche begegnenigt haben sollen.'

In [34]:
de_en_dev[1]

{'translation': {'de': 'Eine Irren-Anstalt, wo sich heute Jugendliche begegnen sollen.',
  'en': 'A mental asylum, where today young people are said to meet.'}}

In [38]:
decode(baseline_translation.flatten().tolist())
# Es war auch ein Anti- Rauch- Aktiv- ist und nahmen an mehreren Kampagnen teil .

'Ein geistiges Asyl, wo heute junge Menschen sich treffen sollen.'

In [ ]:
# compute score using BLEU
def compute_score(prediction, actual):
  pass